In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import roc_curve, precision_recall_curve, average_precision_score, f1_score
from sklearn.model_selection import GroupKFold, GroupShuffleSplit
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
import lightgbm as lgb
import xgboost as xgb
from catboost import Pool, CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

## Данный ноутбук содержит лишь непосредственно код, запуская который можно получить лучшее (по public score) решение и некоторые результаты переборов по сетке. Код перебора и прочее удалён, всё равно никто не будет запускать и ждать

In [2]:
from IPython.core.display import clear_output
import sys
def change_output(x):
    clear_output()
    sys.stdout.write(x)
    sys.stdout.flush()


In [3]:
def load_data(filename, sep=","):
    '''
        Функция для загрузки датасета без инициализации класса. 
        
        Возвращает:
         data - полный DataFrame
         fetures - список признаков для обучения/предсказания
         x - датасет для тренировки/предсказания в формате np.array
         y - целевой признак, если он есть. В случае с тестовым датасетом возвращает None
    '''
    
    data = pd.read_table(filename, sep=sep).dropna() #.fillna(-999999)    
    
    data = data[~(data['chainlen']>1000)]

    if ('DSSR' in data.columns):
        data.drop('DSSR', axis=1, inplace=True)    
    
    features = list(deepcopy(data.columns))
    [features.remove(column) for column in ['Id','index', 'pdb_chain', 'mg'] if column in data.columns];
    x_test = np.array(data[features])
    
    try:
        y_test = np.array(data['mg'])
    except: 
        y_test = None
    change_output('Data loaded')
    return {'data':data, 'features':features, 'x':x_test, 'y':y_test}

In [4]:
test = load_data("test.csv")

Data loaded

In [5]:
change_output('Loading data...')        
data = pd.read_table("train.csv")
change_output('Data processing...')  
if ('DSSR' in data.columns):
    data.drop('DSSR', axis=1, inplace=True)            
data = data.dropna()
y = np.array(np.matrix(data['mg']).flatten().tolist()[0], copy=True) 
data_numpy = np.matrix(data)
features = list(data.columns)
features.remove('pdb_chain')
features.remove('mg')
groups = np.array(data['pdb_chain'])
X = np.array(data[features])
features.append('mg')
change_output('Done')  

Done

In [7]:
X.shape, test["x"].shape

((209043, 383), (4045, 383))

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=17)
clf = 17

In [9]:
%%time
clf = xgb.XGBClassifier(scale_pos_weight=8.255102040816327, n_estimators=600, gamma=10, random_state=17)
clf.fit(X_train, y_train)

CPU times: user 9min 17s, sys: 404 ms, total: 9min 17s
Wall time: 9min 18s


In [10]:
print(f1_score(clf.predict(X_test), y_test))
# 0.3310046910330397 (scale_pos_weight=8.255102040816327, n_estimators=100, gamma=0)
# 0.333  (scale_pos_weight=8.255102040816327, n_estimators=100, gamma=1)
# 0.32618517769235955 (scale_pos_weight=8.255102040816327, n_estimators=100, gamma=5)
# 0.3354528850706893 (scale_pos_weight=8.255102040816327, n_estimators=100, gamma=10)
# 0.3817950956273312 (scale_pos_weight=8.255102040816327, n_estimators=300, gamma=10)
# 0.42455511346246333 (scale_pos_weight=8.255102040816327, n_estimators=600, gamma=10, random_state=17)

0.42455511346246333


/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [14]:
# array([0.00772472, 0.00244328, 0.00176118, 0.238831  , 0.00281096]) RF 100
# array([0.1082544 , 0.11260222, 0.05920281, 0.26041424, 0.09496811]) DT 15
# array([0.0664723 , 0.07678245, 0.0174002 , 0.27320219, 0.03447715]) DT 10
# array([0.03578187, 0.01340897, 0.00581879, 0.00132802, 0.00586142]) CatBoost
# array([0.08434712, 0.05030018, 0.01561858, 0.00582165, 0.02612778]) CatBoost 3k
# array([0.10094637, 0.06313806, 0.02475649, 0.01292024, 0.03391199]) CatBoost 6k -> 0.18848
# ... CatBoost 9k -> 0.20512
# array([0.19595154, 0.19486361, 0.13402873, 0.24864054, 0.17646506]) DT -> 0.20076
# (array([0.15511116, 0.1459787 , 0.09896751, 0.25948915, 0.14122632]), 0.16015456743463227) DT md = 20
# (array([0.09605055, 0.11355198, 0.05333333, 0.07721804, 0.09993754]), 0.0880182881326547) ETC 
# (array([0.08023579, 0.10090881, 0.04891122, 0.06303872, 0.09434263]), 0.07748743541127794) ETC ne = 20

# (array([0.0952381 , 0.05911174, 0.01544269, 0.23348956, 0.04101367]), 0.08885915227716767) lgb ne 1000
# (array([0.08036001, 0.05293441, 0.0295868 , 0.23099507, 0.05625414]), 0.09002608666932135) lgb ne 10000 -> 0.19689
""" LGBClassifier:
[mean: 0.05323, std: 0.10577, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 0.1},
 mean: 0.06368, std: 0.09360, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 2.138775510204082},
 mean: 0.21756, std: 0.04924, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 4.177551020408163},
 mean: 0.28336, std: 0.03686, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 6.216326530612245},
 mean: 0.28418, std: 0.02498, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 8.255102040816327},
 mean: 0.27476, std: 0.01649, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 10.29387755102041},
 mean: 0.26686, std: 0.01281, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 12.332653061224491},
 mean: 0.26051, std: 0.00943, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 14.371428571428572},
 mean: 0.25459, std: 0.00911, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 16.410204081632656},
 mean: 0.25052, std: 0.00854, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 18.44897959183674},
 mean: 0.24771, std: 0.00776, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 20.487755102040822},
 mean: 0.24520, std: 0.00752, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 22.5265306122449},
 mean: 0.24357, std: 0.00746, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 24.565306122448984},
 mean: 0.24230, std: 0.00696, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 26.604081632653067},
 mean: 0.24141, std: 0.00649, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 28.642857142857146},
 mean: 0.24099, std: 0.00645, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 30.68163265306123},
 mean: 0.24033, std: 0.00659, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 32.72040816326531},
 mean: 0.24001, std: 0.00650, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 34.759183673469394},
 mean: 0.23971, std: 0.00622, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 36.79795918367348},
 mean: 0.23942, std: 0.00630, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 38.83673469387756},
 mean: 0.23914, std: 0.00604, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 40.87551020408164},
 mean: 0.23893, std: 0.00613, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 42.91428571428572},
 mean: 0.23895, std: 0.00575, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 44.9530612244898},
 mean: 0.23884, std: 0.00562, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 46.991836734693884},
 mean: 0.23861, std: 0.00563, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 49.030612244897966},
 mean: 0.23875, std: 0.00551, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 51.06938775510205},
 mean: 0.23881, std: 0.00554, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 53.10816326530613},
 mean: 0.23869, std: 0.00550, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 55.146938775510215},
 mean: 0.23872, std: 0.00552, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 57.18571428571429},
 mean: 0.23864, std: 0.00527, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 59.22448979591837},
 mean: 0.23866, std: 0.00518, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 61.263265306122456},
 mean: 0.23858, std: 0.00518, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 63.30204081632654},
 mean: 0.23858, std: 0.00511, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 65.34081632653061},
 mean: 0.23855, std: 0.00496, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 67.37959183673469},
 mean: 0.23852, std: 0.00509, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 69.41836734693878},
 mean: 0.23855, std: 0.00503, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 71.45714285714286},
 mean: 0.23844, std: 0.00495, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 73.49591836734695},
 mean: 0.23836, std: 0.00500, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 75.53469387755102},
 mean: 0.23841, std: 0.00496, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 77.57346938775511},
 mean: 0.23845, std: 0.00485, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 79.61224489795919},
 mean: 0.23834, std: 0.00495, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 81.65102040816328},
 mean: 0.23836, std: 0.00481, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 83.68979591836735},
 mean: 0.23830, std: 0.00479, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 85.72857142857143},
 mean: 0.23834, std: 0.00483, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 87.76734693877552},
 mean: 0.23836, std: 0.00479, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 89.8061224489796},
 mean: 0.23844, std: 0.00481, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 91.84489795918368},
 mean: 0.23827, std: 0.00483, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 93.88367346938776},
 mean: 0.23833, std: 0.00469, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 95.92244897959185},
 mean: 0.23839, std: 0.00475, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 97.96122448979592},
 mean: 0.23829, std: 0.00484, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 100.0}]
 
 
 gscv.best_score_

0.2841823044383953  -> 0.34377

gscv.best_params_

{'learning_rate': 0.01,
 'n_estimators': 300,
 'scale_pos_weight': 8.255102040816327}
 """

""" LGBClassifier:
[mean: 0.20706, std: 0.05033, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 4.0},
 mean: 0.26685, std: 0.04386, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 5.333333333333333},
 mean: 0.28560, std: 0.03334, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 6.666666666666666},
 mean: 0.28459, std: 0.02570, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 8.0},
 mean: 0.27991, std: 0.01930, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 9.333333333333332},
 mean: 0.27388, std: 0.01632, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 10.666666666666666},
 mean: 0.26811, std: 0.01353, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 12.0},
 mean: 0.26414, std: 0.01186, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 13.333333333333332},
 mean: 0.25941, std: 0.00942, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 14.666666666666666},
 mean: 0.25501, std: 0.00953, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 16.0},
 mean: 0.22041, std: 0.04872, params: {'learning_rate': 0.01, 'n_estimators': 500, 'scale_pos_weight': 4.0},
 mean: 0.26731, std: 0.04250, params: {'learning_rate': 0.01, 'n_estimators': 500, 'scale_pos_weight': 5.333333333333333},
 mean: 0.28627, std: 0.03440, params: {'learning_rate': 0.01, 'n_estimators': 500, 'scale_pos_weight': 6.666666666666666},
 mean: 0.28922, std: 0.02897, params: {'learning_rate': 0.01, 'n_estimators': 500, 'scale_pos_weight': 8.0},
 mean: 0.28460, std: 0.02475, params: {'learning_rate': 0.01, 'n_estimators': 500, 'scale_pos_weight': 9.333333333333332},
 mean: 0.28032, std: 0.02167, params: {'learning_rate': 0.01, 'n_estimators': 500, 'scale_pos_weight': 10.666666666666666},
 mean: 0.27551, std: 0.01801, params: {'learning_rate': 0.01, 'n_estimators': 500, 'scale_pos_weight': 12.0},
 mean: 0.27217, std: 0.01669, params: {'learning_rate': 0.01, 'n_estimators': 500, 'scale_pos_weight': 13.333333333333332},
 mean: 0.26754, std: 0.01392, params: {'learning_rate': 0.01, 'n_estimators': 500, 'scale_pos_weight': 14.666666666666666},
 mean: 0.26370, std: 0.01351, params: {'learning_rate': 0.01, 'n_estimators': 500, 'scale_pos_weight': 16.0},
 mean: 0.22822, std: 0.04405, params: {'learning_rate': 0.01, 'n_estimators': 1000, 'scale_pos_weight': 4.0},
 mean: 0.26802, std: 0.03971, params: {'learning_rate': 0.01, 'n_estimators': 1000, 'scale_pos_weight': 5.333333333333333},
 mean: 0.28289, std: 0.03304, params: {'learning_rate': 0.01, 'n_estimators': 1000, 'scale_pos_weight': 6.666666666666666},
 mean: 0.28963, std: 0.02979, params: {'learning_rate': 0.01, 'n_estimators': 1000, 'scale_pos_weight': 8.0},
 mean: 0.28981, std: 0.02785, params: {'learning_rate': 0.01, 'n_estimators': 1000, 'scale_pos_weight': 9.333333333333332},
 mean: 0.28772, std: 0.02674, params: {'learning_rate': 0.01, 'n_estimators': 1000, 'scale_pos_weight': 10.666666666666666},
 mean: 0.28451, std: 0.02468, params: {'learning_rate': 0.01, 'n_estimators': 1000, 'scale_pos_weight': 12.0},
 mean: 0.28119, std: 0.02152, params: {'learning_rate': 0.01, 'n_estimators': 1000, 'scale_pos_weight': 13.333333333333332},
 mean: 0.27769, std: 0.01985, params: {'learning_rate': 0.01, 'n_estimators': 1000, 'scale_pos_weight': 14.666666666666666},
 mean: 0.27465, std: 0.01897, params: {'learning_rate': 0.01, 'n_estimators': 1000, 'scale_pos_weight': 16.0},
 mean: 0.20395, std: 0.04719, params: {'learning_rate': 0.01, 'n_estimators': 5000, 'scale_pos_weight': 4.0},
 mean: 0.23063, std: 0.03735, params: {'learning_rate': 0.01, 'n_estimators': 5000, 'scale_pos_weight': 5.333333333333333},
 mean: 0.25114, std: 0.03337, params: {'learning_rate': 0.01, 'n_estimators': 5000, 'scale_pos_weight': 6.666666666666666},
 mean: 0.26193, std: 0.02488, params: {'learning_rate': 0.01, 'n_estimators': 5000, 'scale_pos_weight': 8.0},
 mean: 0.27229, std: 0.02798, params: {'learning_rate': 0.01, 'n_estimators': 5000, 'scale_pos_weight': 9.333333333333332},
 mean: 0.27565, std: 0.02612, params: {'learning_rate': 0.01, 'n_estimators': 5000, 'scale_pos_weight': 10.666666666666666},
 mean: 0.27785, std: 0.02704, params: {'learning_rate': 0.01, 'n_estimators': 5000, 'scale_pos_weight': 12.0},
 mean: 0.27886, std: 0.02668, params: {'learning_rate': 0.01, 'n_estimators': 5000, 'scale_pos_weight': 13.333333333333332},
 mean: 0.27799, std: 0.02680, params: {'learning_rate': 0.01, 'n_estimators': 5000, 'scale_pos_weight': 14.666666666666666},
 mean: 0.28051, std: 0.02716, params: {'learning_rate': 0.01, 'n_estimators': 5000, 'scale_pos_weight': 16.0},
 mean: 0.23492, std: 0.04060, params: {'learning_rate': 0.05, 'n_estimators': 300, 'scale_pos_weight': 4.0},
 mean: 0.26552, std: 0.03649, params: {'learning_rate': 0.05, 'n_estimators': 300, 'scale_pos_weight': 5.333333333333333},
 mean: 0.28126, std: 0.03310, params: {'learning_rate': 0.05, 'n_estimators': 300, 'scale_pos_weight': 6.666666666666666},
 mean: 0.28760, std: 0.03033, params: {'learning_rate': 0.05, 'n_estimators': 300, 'scale_pos_weight': 8.0},
 mean: 0.28598, std: 0.03007, params: {'learning_rate': 0.05, 'n_estimators': 300, 'scale_pos_weight': 9.333333333333332},
 mean: 0.28809, std: 0.02763, params: {'learning_rate': 0.05, 'n_estimators': 300, 'scale_pos_weight': 10.666666666666666},
 mean: 0.28643, std: 0.02514, params: {'learning_rate': 0.05, 'n_estimators': 300, 'scale_pos_weight': 12.0},
 mean: 0.28320, std: 0.02437, params: {'learning_rate': 0.05, 'n_estimators': 300, 'scale_pos_weight': 13.333333333333332},
 mean: 0.27911, std: 0.02237, params: {'learning_rate': 0.05, 'n_estimators': 300, 'scale_pos_weight': 14.666666666666666},
 mean: 0.27837, std: 0.02229, params: {'learning_rate': 0.05, 'n_estimators': 300, 'scale_pos_weight': 16.0},
 mean: 0.22883, std: 0.04080, params: {'learning_rate': 0.05, 'n_estimators': 500, 'scale_pos_weight': 4.0},
 mean: 0.25795, std: 0.03532, params: {'learning_rate': 0.05, 'n_estimators': 500, 'scale_pos_weight': 5.333333333333333},
 mean: 0.27354, std: 0.03411, params: {'learning_rate': 0.05, 'n_estimators': 500, 'scale_pos_weight': 6.666666666666666},
 mean: 0.28370, std: 0.03061, params: {'learning_rate': 0.05, 'n_estimators': 500, 'scale_pos_weight': 8.0},
 mean: 0.28281, std: 0.03125, params: {'learning_rate': 0.05, 'n_estimators': 500, 'scale_pos_weight': 9.333333333333332},
 mean: 0.28486, std: 0.02903, params: {'learning_rate': 0.05, 'n_estimators': 500, 'scale_pos_weight': 10.666666666666666},
 mean: 0.28626, std: 0.02596, params: {'learning_rate': 0.05, 'n_estimators': 500, 'scale_pos_weight': 12.0},
 mean: 0.28394, std: 0.02681, params: {'learning_rate': 0.05, 'n_estimators': 500, 'scale_pos_weight': 13.333333333333332},
 mean: 0.28263, std: 0.02496, params: {'learning_rate': 0.05, 'n_estimators': 500, 'scale_pos_weight': 14.666666666666666},
 mean: 0.28228, std: 0.02642, params: {'learning_rate': 0.05, 'n_estimators': 500, 'scale_pos_weight': 16.0},
 mean: 0.20808, std: 0.04105, params: {'learning_rate': 0.05, 'n_estimators': 1000, 'scale_pos_weight': 4.0},
 mean: 0.23557, std: 0.03610, params: {'learning_rate': 0.05, 'n_estimators': 1000, 'scale_pos_weight': 5.333333333333333},
 mean: 0.25194, std: 0.03524, params: {'learning_rate': 0.05, 'n_estimators': 1000, 'scale_pos_weight': 6.666666666666666},
 mean: 0.26022, std: 0.03107, params: {'learning_rate': 0.05, 'n_estimators': 1000, 'scale_pos_weight': 8.0},
 mean: 0.26884, std: 0.03023, params: {'learning_rate': 0.05, 'n_estimators': 1000, 'scale_pos_weight': 9.333333333333332},
 mean: 0.27357, std: 0.02886, params: {'learning_rate': 0.05, 'n_estimators': 1000, 'scale_pos_weight': 10.666666666666666},
 mean: 0.27729, std: 0.02388, params: {'learning_rate': 0.05, 'n_estimators': 1000, 'scale_pos_weight': 12.0},
 mean: 0.27789, std: 0.02748, params: {'learning_rate': 0.05, 'n_estimators': 1000, 'scale_pos_weight': 13.333333333333332},
 mean: 0.27606, std: 0.02205, params: {'learning_rate': 0.05, 'n_estimators': 1000, 'scale_pos_weight': 14.666666666666666},
 mean: 0.27736, std: 0.02499, params: {'learning_rate': 0.05, 'n_estimators': 1000, 'scale_pos_weight': 16.0},
 mean: 0.14304, std: 0.05671, params: {'learning_rate': 0.05, 'n_estimators': 5000, 'scale_pos_weight': 4.0},
 mean: 0.15952, std: 0.05062, params: {'learning_rate': 0.05, 'n_estimators': 5000, 'scale_pos_weight': 5.333333333333333},
 mean: 0.17057, std: 0.04808, params: {'learning_rate': 0.05, 'n_estimators': 5000, 'scale_pos_weight': 6.666666666666666},
 mean: 0.18220, std: 0.04403, params: {'learning_rate': 0.05, 'n_estimators': 5000, 'scale_pos_weight': 8.0},
 mean: 0.18981, std: 0.04020, params: {'learning_rate': 0.05, 'n_estimators': 5000, 'scale_pos_weight': 9.333333333333332},
 mean: 0.19500, std: 0.03806, params: {'learning_rate': 0.05, 'n_estimators': 5000, 'scale_pos_weight': 10.666666666666666},
 mean: 0.20141, std: 0.03955, params: {'learning_rate': 0.05, 'n_estimators': 5000, 'scale_pos_weight': 12.0},
 mean: 0.20449, std: 0.03588, params: {'learning_rate': 0.05, 'n_estimators': 5000, 'scale_pos_weight': 13.333333333333332},
 mean: 0.20733, std: 0.03551, params: {'learning_rate': 0.05, 'n_estimators': 5000, 'scale_pos_weight': 14.666666666666666},
 mean: 0.21532, std: 0.03224, params: {'learning_rate': 0.05, 'n_estimators': 5000, 'scale_pos_weight': 16.0},
 mean: 0.22094, std: 0.04181, params: {'learning_rate': 0.1, 'n_estimators': 300, 'scale_pos_weight': 4.0},
 mean: 0.25424, std: 0.03253, params: {'learning_rate': 0.1, 'n_estimators': 300, 'scale_pos_weight': 5.333333333333333},
 mean: 0.26768, std: 0.03290, params: {'learning_rate': 0.1, 'n_estimators': 300, 'scale_pos_weight': 6.666666666666666},
 mean: 0.27622, std: 0.03376, params: {'learning_rate': 0.1, 'n_estimators': 300, 'scale_pos_weight': 8.0},
 mean: 0.28154, std: 0.02565, params: {'learning_rate': 0.1, 'n_estimators': 300, 'scale_pos_weight': 9.333333333333332},
 mean: 0.28383, std: 0.02564, params: {'learning_rate': 0.1, 'n_estimators': 300, 'scale_pos_weight': 10.666666666666666},
 mean: 0.28273, std: 0.02643, params: {'learning_rate': 0.1, 'n_estimators': 300, 'scale_pos_weight': 12.0},
 mean: 0.28080, std: 0.02688, params: {'learning_rate': 0.1, 'n_estimators': 300, 'scale_pos_weight': 13.333333333333332},
 mean: 0.27759, std: 0.02513, params: {'learning_rate': 0.1, 'n_estimators': 300, 'scale_pos_weight': 14.666666666666666},
 mean: 0.28004, std: 0.02120, params: {'learning_rate': 0.1, 'n_estimators': 300, 'scale_pos_weight': 16.0},
 mean: 0.20670, std: 0.04570, params: {'learning_rate': 0.1, 'n_estimators': 500, 'scale_pos_weight': 4.0},
 mean: 0.23650, std: 0.03530, params: {'learning_rate': 0.1, 'n_estimators': 500, 'scale_pos_weight': 5.333333333333333},
 mean: 0.25050, std: 0.03375, params: {'learning_rate': 0.1, 'n_estimators': 500, 'scale_pos_weight': 6.666666666666666},
 mean: 0.26208, std: 0.03096, params: {'learning_rate': 0.1, 'n_estimators': 500, 'scale_pos_weight': 8.0},
 mean: 0.26656, std: 0.02353, params: {'learning_rate': 0.1, 'n_estimators': 500, 'scale_pos_weight': 9.333333333333332},
 mean: 0.27131, std: 0.02730, params: {'learning_rate': 0.1, 'n_estimators': 500, 'scale_pos_weight': 10.666666666666666},
 mean: 0.27289, std: 0.02757, params: {'learning_rate': 0.1, 'n_estimators': 500, 'scale_pos_weight': 12.0},
 mean: 0.27570, std: 0.02684, params: {'learning_rate': 0.1, 'n_estimators': 500, 'scale_pos_weight': 13.333333333333332},
 mean: 0.27408, std: 0.02351, params: {'learning_rate': 0.1, 'n_estimators': 500, 'scale_pos_weight': 14.666666666666666},
 mean: 0.27256, std: 0.02482, params: {'learning_rate': 0.1, 'n_estimators': 500, 'scale_pos_weight': 16.0},
 mean: 0.18596, std: 0.04930, params: {'learning_rate': 0.1, 'n_estimators': 1000, 'scale_pos_weight': 4.0},
 mean: 0.20549, std: 0.03619, params: {'learning_rate': 0.1, 'n_estimators': 1000, 'scale_pos_weight': 5.333333333333333},
 mean: 0.22255, std: 0.03728, params: {'learning_rate': 0.1, 'n_estimators': 1000, 'scale_pos_weight': 6.666666666666666},
 mean: 0.23551, std: 0.03574, params: {'learning_rate': 0.1, 'n_estimators': 1000, 'scale_pos_weight': 8.0},
 mean: 0.24121, std: 0.03051, params: {'learning_rate': 0.1, 'n_estimators': 1000, 'scale_pos_weight': 9.333333333333332},
 mean: 0.24589, std: 0.02480, params: {'learning_rate': 0.1, 'n_estimators': 1000, 'scale_pos_weight': 10.666666666666666},
 mean: 0.24879, std: 0.02580, params: {'learning_rate': 0.1, 'n_estimators': 1000, 'scale_pos_weight': 12.0},
 mean: 0.25603, std: 0.03071, params: {'learning_rate': 0.1, 'n_estimators': 1000, 'scale_pos_weight': 13.333333333333332},
 mean: 0.25319, std: 0.02462, params: {'learning_rate': 0.1, 'n_estimators': 1000, 'scale_pos_weight': 14.666666666666666},
 mean: 0.25446, std: 0.02384, params: {'learning_rate': 0.1, 'n_estimators': 1000, 'scale_pos_weight': 16.0},
 mean: 0.12828, std: 0.06079, params: {'learning_rate': 0.1, 'n_estimators': 5000, 'scale_pos_weight': 4.0},
 mean: 0.13462, std: 0.05532, params: {'learning_rate': 0.1, 'n_estimators': 5000, 'scale_pos_weight': 5.333333333333333},
 mean: 0.14255, std: 0.05321, params: {'learning_rate': 0.1, 'n_estimators': 5000, 'scale_pos_weight': 6.666666666666666},
 mean: 0.15224, std: 0.05075, params: {'learning_rate': 0.1, 'n_estimators': 5000, 'scale_pos_weight': 8.0},
 mean: 0.15668, std: 0.05082, params: {'learning_rate': 0.1, 'n_estimators': 5000, 'scale_pos_weight': 9.333333333333332},
 mean: 0.15897, std: 0.04729, params: {'learning_rate': 0.1, 'n_estimators': 5000, 'scale_pos_weight': 10.666666666666666},
 mean: 0.16672, std: 0.04653, params: {'learning_rate': 0.1, 'n_estimators': 5000, 'scale_pos_weight': 12.0},
 mean: 0.17135, std: 0.04642, params: {'learning_rate': 0.1, 'n_estimators': 5000, 'scale_pos_weight': 13.333333333333332},
 mean: 0.17110, std: 0.04322, params: {'learning_rate': 0.1, 'n_estimators': 5000, 'scale_pos_weight': 14.666666666666666},
 mean: 0.16905, std: 0.04374, params: {'learning_rate': 0.1, 'n_estimators': 5000, 'scale_pos_weight': 16.0},
 mean: 0.19312, std: 0.04069, params: {'learning_rate': 0.5, 'n_estimators': 300, 'scale_pos_weight': 4.0},
 mean: 0.21341, std: 0.03160, params: {'learning_rate': 0.5, 'n_estimators': 300, 'scale_pos_weight': 5.333333333333333},
 mean: 0.22072, std: 0.03369, params: {'learning_rate': 0.5, 'n_estimators': 300, 'scale_pos_weight': 6.666666666666666},
 mean: 0.21744, std: 0.03032, params: {'learning_rate': 0.5, 'n_estimators': 300, 'scale_pos_weight': 8.0},
 mean: 0.22477, std: 0.02650, params: {'learning_rate': 0.5, 'n_estimators': 300, 'scale_pos_weight': 9.333333333333332},
 mean: 0.22657, std: 0.02755, params: {'learning_rate': 0.5, 'n_estimators': 300, 'scale_pos_weight': 10.666666666666666},
 mean: 0.23377, std: 0.03099, params: {'learning_rate': 0.5, 'n_estimators': 300, 'scale_pos_weight': 12.0},
 mean: 0.23871, std: 0.02639, params: {'learning_rate': 0.5, 'n_estimators': 300, 'scale_pos_weight': 13.333333333333332},
 mean: 0.22775, std: 0.02453, params: {'learning_rate': 0.5, 'n_estimators': 300, 'scale_pos_weight': 14.666666666666666},
 mean: 0.23320, std: 0.02443, params: {'learning_rate': 0.5, 'n_estimators': 300, 'scale_pos_weight': 16.0},
 mean: 0.18128, std: 0.04470, params: {'learning_rate': 0.5, 'n_estimators': 500, 'scale_pos_weight': 4.0},
 mean: 0.19416, std: 0.03488, params: {'learning_rate': 0.5, 'n_estimators': 500, 'scale_pos_weight': 5.333333333333333},
 mean: 0.20412, std: 0.03998, params: {'learning_rate': 0.5, 'n_estimators': 500, 'scale_pos_weight': 6.666666666666666},
 mean: 0.20243, std: 0.03347, params: {'learning_rate': 0.5, 'n_estimators': 500, 'scale_pos_weight': 8.0},
 mean: 0.20433, std: 0.03390, params: {'learning_rate': 0.5, 'n_estimators': 500, 'scale_pos_weight': 9.333333333333332},
 mean: 0.20763, std: 0.03218, params: {'learning_rate': 0.5, 'n_estimators': 500, 'scale_pos_weight': 10.666666666666666},
 mean: 0.21488, std: 0.03639, params: {'learning_rate': 0.5, 'n_estimators': 500, 'scale_pos_weight': 12.0},
 mean: 0.22009, std: 0.03097, params: {'learning_rate': 0.5, 'n_estimators': 500, 'scale_pos_weight': 13.333333333333332},
 mean: 0.21002, std: 0.03417, params: {'learning_rate': 0.5, 'n_estimators': 500, 'scale_pos_weight': 14.666666666666666},
 mean: 0.21678, std: 0.02962, params: {'learning_rate': 0.5, 'n_estimators': 500, 'scale_pos_weight': 16.0},
 mean: 0.15891, std: 0.05075, params: {'learning_rate': 0.5, 'n_estimators': 1000, 'scale_pos_weight': 4.0},
 mean: 0.16957, std: 0.04363, params: {'learning_rate': 0.5, 'n_estimators': 1000, 'scale_pos_weight': 5.333333333333333},
 mean: 0.17904, std: 0.04641, params: {'learning_rate': 0.5, 'n_estimators': 1000, 'scale_pos_weight': 6.666666666666666},
 mean: 0.17634, std: 0.04313, params: {'learning_rate': 0.5, 'n_estimators': 1000, 'scale_pos_weight': 8.0},
 mean: 0.18070, std: 0.04215, params: {'learning_rate': 0.5, 'n_estimators': 1000, 'scale_pos_weight': 9.333333333333332},
 mean: 0.18228, std: 0.04064, params: {'learning_rate': 0.5, 'n_estimators': 1000, 'scale_pos_weight': 10.666666666666666},
 mean: 0.18575, std: 0.04308, params: {'learning_rate': 0.5, 'n_estimators': 1000, 'scale_pos_weight': 12.0},
 mean: 0.18909, std: 0.04002, params: {'learning_rate': 0.5, 'n_estimators': 1000, 'scale_pos_weight': 13.333333333333332},
 mean: 0.18546, std: 0.04122, params: {'learning_rate': 0.5, 'n_estimators': 1000, 'scale_pos_weight': 14.666666666666666},
 mean: 0.18941, std: 0.03956, params: {'learning_rate': 0.5, 'n_estimators': 1000, 'scale_pos_weight': 16.0},
 mean: 0.11905, std: 0.06393, params: {'learning_rate': 0.5, 'n_estimators': 5000, 'scale_pos_weight': 4.0},
 mean: 0.12714, std: 0.05933, params: {'learning_rate': 0.5, 'n_estimators': 5000, 'scale_pos_weight': 5.333333333333333},
 mean: 0.12654, std: 0.06465, params: {'learning_rate': 0.5, 'n_estimators': 5000, 'scale_pos_weight': 6.666666666666666},
 mean: 0.12593, std: 0.06096, params: {'learning_rate': 0.5, 'n_estimators': 5000, 'scale_pos_weight': 8.0},
 mean: 0.13105, std: 0.05843, params: {'learning_rate': 0.5, 'n_estimators': 5000, 'scale_pos_weight': 9.333333333333332},
 mean: 0.12967, std: 0.05950, params: {'learning_rate': 0.5, 'n_estimators': 5000, 'scale_pos_weight': 10.666666666666666},
 mean: 0.13478, std: 0.05601, params: {'learning_rate': 0.5, 'n_estimators': 5000, 'scale_pos_weight': 12.0},
 mean: 0.13623, std: 0.05727, params: {'learning_rate': 0.5, 'n_estimators': 5000, 'scale_pos_weight': 13.333333333333332},
 mean: 0.13384, std: 0.05809, params: {'learning_rate': 0.5, 'n_estimators': 5000, 'scale_pos_weight': 14.666666666666666},
 mean: 0.13661, std: 0.05929, params: {'learning_rate': 0.5, 'n_estimators': 5000, 'scale_pos_weight': 16.0}]

gscv.best_score_

0.2898096239376933

gscv.best_params_

{'learning_rate': 0.01,
 'n_estimators': 1000,
 'scale_pos_weight': 9.333333333333332}
"""

""" LGBClassifier:
[mean: 0.28876, std: 0.02667, params: {'boosting_type': 'gbdt', 'learning_rate': 0.01, 'n_estimators': 750, 'scale_pos_weight': 9.0},
 mean:0.27868, std: 0.01830, params: {'boosting_type': 'dart', 'learning_rate': 0.01, 'n_estimators': 750, 'scale_pos_weight': 9.0},
 mean: 0.29242, std: 0.02722, params: {'boosting_type': 'goss', 'learning_rate': 0.01, 'n_estimators': 750, 'scale_pos_weight': 9.0}]

gscv.best_score_

0.29242072490189625 --> 0.33958

gscv.best_params_

{'boosting_type': 'goss',
 'learning_rate': 0.01,
 'n_estimators': 750,
 'scale_pos_weight': 9.0}
"""
""""""

''

In [19]:
clf

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=10, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=600,
       n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=17, reg_alpha=0, reg_lambda=1,
       scale_pos_weight=8.255102040816327, seed=None, silent=True,
       subsample=1)

In [20]:
prediction = clf.predict(test["x"]).astype(int)

/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [21]:
np.unique(prediction, return_counts=True)

(array([0, 1]), array([3288,  757]))

In [22]:
df_sub = pd.read_csv("sample_submission.csv")
df_sub["mg"] = prediction
df_sub.to_csv("my_sub.csv", index=False)